In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import xlsxwriter

# 📌 Function to load MRI images
def load_mri_images(directory, probe):
    """
    Loads MRI images from a specified directory and converts them to numpy arrays.
    """
    if not os.path.exists(directory):
        print(f"❌ ERROR: The directory {directory} does not exist! Please check the path.")
        return {}

    images = {}
    probe_prefix = "711" if probe == "p2" else "712"  # 711 → 3FDGal, 712 → 3FDG

    for file_name in os.listdir(directory):
        if file_name.startswith(probe_prefix) and file_name.endswith(".tif"):
            file_path = os.path.join(directory, file_name)
            try:
                img = Image.open(file_path)
                img_array = np.array(img)
                images[file_name] = img_array
            except Exception as e:
                print(f"❌ ERROR loading {file_path}: {e}")

    if not images:
        print(f"⚠ No images found for probe {probe.upper()} in {directory}!")

    return images

# 📌 **Updated function to compute SNR (Matching the visualization code)**
def compute_snr(image):
    """
    Computes the Signal-to-Noise Ratio (SNR) for a given MRI image.

    📌 Uses **same** signal & noise regions as visualization function:
    - **Signal Region:** Pre-defined based on expected fluorinated sugar activity.
    - **Noise Region:** Selected from a stable, non-activated background.
    """
    signal_region = image[7:24, 10:20]  # Defined signal region
    noise_region = image[2:31, 2:4]  # Defined noise region

    noise = np.std(noise_region)
    max_signal = np.max(signal_region)
    snr = max_signal / noise if noise > 0 else 0

    return snr, max_signal, noise

# 📌 Function to save SNR results to an Excel file
def save_snr_to_excel(images, probe, repo_directory):
    """
    Computes and saves SNR values into an Excel file, using **same SNR calculation** as visualization.
    """
    snr_results_dir = os.path.join(repo_directory, "snr_results")
    os.makedirs(snr_results_dir, exist_ok=True)

    metabolite_map = {
        "p1": {"m1": "3FDGlucose", "m3": "3FDGluconic Acid", "m4": "3FDSorbitol"},
        "p2": {"m1": "3FDGalactose", "m3": "3FDGalactonic Acid", "m4": "3FDGalactitol"}
    }
    time_map = {"t1": "1hr", "t2": "2hr", "t3": "3hr"}

    summary_data = []

    for file_path, img in images.items():
        snr, max_signal, noise = compute_snr(np.array(img))  # ✅ **Now using the correct SNR function!**

        parts = file_path.replace(".tif", "").split("_")
        if len(parts) < 4:
            continue  # Skip unexpected formats

        time_label = time_map.get(parts[2], parts[2])
        metabolite_label = metabolite_map[probe].get(parts[3], parts[3])

        summary_data.append([file_path, metabolite_label, time_label, snr, max_signal, noise])

    output_excel = os.path.join(snr_results_dir, f"SNR_Results_{probe.upper()}.xlsx")
    df = pd.DataFrame(summary_data, columns=["Filename", "Metabolite", "Time", "SNR", "Max Signal", "Noise Std Dev"])
    
    with pd.ExcelWriter(output_excel, engine="xlsxwriter") as writer:
        df.to_excel(writer, sheet_name="SNR_Data", index=False)

    print(f"✅ SNR results saved successfully: {output_excel}")

# 📝 Define repository path
repo_directory = r"YOUR_LOCAL_PATH_HERE/FMRI-Metabolism-Quantification-main" # ← UPDATE THIS PATH!
example_images_dir = os.path.join(repo_directory, "example_images_3FDGal")  # Change to "example_images_3FDGal" for P2
probe = "p2"  # "p1" for 3FDG, "p2" for 3FDGal

# 📂 Load images before calling save_snr_to_excel
images = load_mri_images(example_images_dir, probe)

# ✅ Check if images loaded correctly
if not images:
    raise ValueError("❌ ERROR: No images were loaded! Check the directory path and probe type.")

# 🔍 Compute and save SNR results to Excel
save_snr_to_excel(images, probe, repo_directory)
